In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

C:\Users\Abhishek\Anaconda2\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
# load training and test data
train = pd.read_csv('data/train_processed.csv')
test = pd.read_csv('data/test_processed.csv')

In [3]:
# preprocess skills feature

# determine unique skills
skills = set()
for m in train.skills:
    skills.update(g for g in m.split('|'))
skills = sorted(skills)

#make a column for each skill
for skill in skills:
    train[skill] = [int(skill in s.split('|')) for s in train.skills]

In [4]:
# do same calculation for test set as well
for skill in skills:
    test[skill] = [int(skill in s.split('|')) for s in test.skills]

In [5]:
# balanced dataset
%run scripts/helper.py
train = balanced_sample(train)

In [6]:
# response variable
y = train.solved_status

## Prepare a very simple model based on just the accuracy of the problem

In [7]:
%run scripts/features.py
%run scripts/models.py
%run scripts/eval.py

In [8]:
model= build_extreme_gradient_boosting(train, test)
# model = build_random_forest_classifier(train, test)
# model = build_logistic_regression_model(train, test)
# model = build_knn_classifier(train, test)
# model = build_extra_trees_classifier(train, test)

In [9]:
scores = eval_models([model], train, y)

numeric features shape  (29304L, 8L)
skill features shape  (29304L, 26L)
accuracy score: 0.803552
combined score: 0.803552
numeric features shape  (29304L, 8L)
skill features shape  (29304L, 26L)
accuracy score: 0.805144
combined score: 0.805144
numeric features shape  (29304L, 8L)
skill features shape  (29304L, 26L)
accuracy score: 0.803950
combined score: 0.803950
numeric features shape  (29304L, 8L)
skill features shape  (29304L, 26L)
accuracy score: 0.806020
combined score: 0.806020
numeric features shape  (29304L, 8L)
skill features shape  (29304L, 26L)
accuracy score: 0.803392
combined score: 0.803392



In [10]:
print 'Mean: %f and Standard Deviation: %f' %(np.mean(scores), np.std(scores))

Mean: 0.804412 and Standard Deviation: 0.001012


In [ ]:
predsTrain, predsTest, y_train, y_test = analyze_output(train)

In [ ]:
# confusion matrix
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, predsTest)

In [ ]:
misclassified = train.ix[(y_test != predsTest).index.values]
misclassified['preds'] = predsTest

In [ ]:
misclassified_class_0 = misclassified[misclassified.solved_status == 0]

In [ ]:
misclassified_class_0[['accuracy', 'solved_status', 'preds']]

In [ ]:
sns.pointplot(x="solved_status", y="accuracy", data=misclassified);

In [11]:
# train model
import time

start_time = time.time()
# model1.fit(train, y)
# model2.fit(train, y)
model.fit(train, y)
elapsed_time = time.time() - start_time

print 'It took %f seconds to train the model ' %(elapsed_time)

numeric features shape  (41862L, 8L)
skill features shape  (41862L, 26L)
It took 16.569000 seconds to train the model 


In [12]:
# predictions1  = model1.predict(test)
# predictions2 = model2.predict(test)

# ensemble_preds = np.array(majority_voting([predictions1, predictions2]))
predictions = model.predict(test)

In [13]:
# submissions
submission_df = pd.DataFrame({'Id': test.id.values, 'solved_status': predictions.astype(int)})
submission_df.to_csv('submissions/twenty_eigth_xgb_one_hot_encoded_skills.csv', index=False)